In [1]:
import pandas as pd
import numpy as np
import re, string
from scipy.stats import zscore
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics, svm
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
!wget http://infolab.tamu.edu/data/social_honeypot_icwsm_2011.zip

--2022-10-10 16:26:17--  http://infolab.tamu.edu/data/social_honeypot_icwsm_2011.zip
Resolving infolab.tamu.edu (infolab.tamu.edu)... 45.55.217.29
Connecting to infolab.tamu.edu (infolab.tamu.edu)|45.55.217.29|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://infolab.tamu.edu/data/social_honeypot_icwsm_2011.zip [following]
--2022-10-10 16:26:17--  https://infolab.tamu.edu/data/social_honeypot_icwsm_2011.zip
Connecting to infolab.tamu.edu (infolab.tamu.edu)|45.55.217.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 263659672 (251M) [application/zip]
Saving to: ‘social_honeypot_icwsm_2011.zip.1’

social_honeypot_icw 100%[===================>] 251.45M  78.4MB/s    in 3.2s    

2022-10-10 16:26:20 (78.4 MB/s) - ‘social_honeypot_icwsm_2011.zip.1’ saved [263659672/263659672]



In [ ]:
!unzip social_honeypot_icwsm_2011.zip

Archive:  social_honeypot_icwsm_2011.zip
replace social_honeypot_icwsm_2011/content_polluters.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: social_honeypot_icwsm_2011/content_polluters.txt  
replace __MACOSX/social_honeypot_icwsm_2011/._content_polluters.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace __MACOSX/social_honeypot_icwsm_2011/._content_polluters.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/social_honeypot_icwsm_2011/._content_polluters.txt  
replace social_honeypot_icwsm_2011/content_polluters_followings.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: social_honeypot_icwsm_2011/content_polluters_followings.txt  
replace __MACOSX/social_honeypot_icwsm_2011/._content_polluters_followings.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/social_honeypot_icwsm_2011/._content_polluters_followings.txt  
replace social_honeypot_icwsm_2011/content_polluters_tweets.txt? [y]es, [n]o, [A]ll, 

In [ ]:
polluters = pd.read_csv('social_honeypot_icwsm_2011/content_polluters.txt',sep='\t', names=
                 ["UserID","CreatedAt","CollectedAt","NumberOfFollowings","NumberOfFollowers",
                  "NumberOfTweets","LengthOfScreenName","LengthOfDescriptionInUserProfile"],
                parse_dates=["CreatedAt","CollectedAt"],index_col=0)

legitimate_users = pd.read_csv('social_honeypot_icwsm_2011/legitimate_users.txt',sep='\t', names=
                 ["UserID","CreatedAt","CollectedAt","NumberOfFollowings","NumberOfFollowers",
                  "NumberOfTweets","LengthOfScreenName","LengthOfDescriptionInUserProfile"],
                parse_dates=["CreatedAt","CollectedAt"],index_col=0)

polluters_tweets = pd.read_csv('social_honeypot_icwsm_2011/content_polluters_tweets.txt',sep='\t', names=
                 ["UserID","TweetID","Tweet","CreatedAt"], parse_dates=["CreatedAt"],index_col=0)

legitimate_users_tweets = pd.read_csv('social_honeypot_icwsm_2011/legitimate_users_tweets.txt',sep='\t', names=
                 ["UserID","TweetID","Tweet","CreatedAt"], parse_dates=["CreatedAt"],index_col=0)


Extract Features

Basic Features

---



Number of Followings

In [ ]:
def noFollowings(data_frame) :
  df = pd.DataFrame({"NumberOfFollowings":data_frame["NumberOfFollowings"]})
  return df

Number of Followers

In [ ]:
def noFollowers(data_frame) :
  df = pd.DataFrame({"NumberOfFollowers":data_frame["NumberOfFollowers"]})
  return df

Number of Tweets

In [ ]:
def noTweets(data_frame) :
  df = pd.DataFrame({"NumberOfTweets": data_frame["NumberOfTweets"] })
  return df

Length of Name

In [ ]:
def nameLength(data_frame) :
  df = pd.DataFrame({"LengthOfScreenName":data_frame["LengthOfScreenName"]})
  return df

Length of Description (Bio)

In [ ]:
def bioLength(data_frame) :
  df = pd.DataFrame({"LengthOfDescription":data_frame["LengthOfDescriptionInUserProfile"]})
  return df

Derived Features

---



Tweets with URL

In [ ]:
def existURL(tweet) :
  if(len(re.findall('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', str(tweet)))==0):
    return 0;
  else:
    return 1;
  
def urlFilter(data_frame) :
  dftemp=pd.DataFrame({"AvgTweetsWithURL": data_frame["Tweet"].apply(existURL)})
  df = dftemp.groupby(["UserID"]).mean()
  return df


Tweets Frequency

In [ ]:
def URLcount(tweet):
  return len(re.findall('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', str(tweet)))

def frequency(data_frame) :
  dftemp=pd.DataFrame({"TweetFrequency": data_frame["Tweet"].apply(URLcount)})
  df = dftemp.groupby(["UserID"]).mean()
  return df

Average Mention Times

In [ ]:
def Atcount(tweet):
  return len(re.findall('\s([@#][\w_-]+)', str(tweet)))

def mentions(data_frame):
  dftemp=pd.DataFrame({"AvgMentionsPerTweet": data_frame["Tweet"].apply(Atcount)})
  df = dftemp.groupby(["UserID"]).mean()
  return df

Average Duration

In [ ]:
def avgDuration(data_frame):
  filter=pd.DataFrame({"CreatedAt":data_frame["CreatedAt"]}).sort_values(by=['UserID','CreatedAt'])
  df_t=pd.DataFrame({"PreviousTweet":filter.groupby(['UserID']).apply(lambda x:x)['CreatedAt'],
                       "NextTweet":filter.groupby(['UserID']).shift(-1)['CreatedAt']})
  df=pd.DataFrame({"avgDuration": (df_t["NextTweet"]-df_t["PreviousTweet"]) / np.timedelta64(1, 'm')})
  df=df.groupby(["UserID"]).mean()
  return df

Max Duration


In [ ]:
def maxDuration(data_frame):
  filter=pd.DataFrame({"CreatedAt":data_frame["CreatedAt"]}).sort_values(by=['UserID','CreatedAt'])
  df_t=pd.DataFrame({"PreviousTweet":filter.groupby(['UserID']).apply(lambda x:x)['CreatedAt'],
                       "NextTweet":filter.groupby(['UserID']).shift(-1)['CreatedAt']})
  df=pd.DataFrame({"maxDuration": (df_t["NextTweet"]-df_t["PreviousTweet"]) / np.timedelta64(1, 'm')})
  df=df.groupby(["UserID"]).max()
  return df

Generate Dataset

In [ ]:
data_set_polluters=pd.concat([urlFilter(polluters_tweets),frequency(polluters_tweets),nameLength(polluters),bioLength(polluters),noFollowings(polluters),
                              mentions(polluters_tweets),noFollowers(polluters),noTweets(polluters),avgDuration(polluters_tweets),maxDuration(polluters_tweets)],axis=1)

data_set_legitimate=pd.concat([urlFilter(legitimate_users_tweets),frequency(legitimate_users_tweets),nameLength(legitimate_users),bioLength(legitimate_users),
                              mentions(legitimate_users_tweets),noFollowings(legitimate_users),noFollowers(legitimate_users),noTweets(legitimate_users),
                              avgDuration(legitimate_users_tweets),maxDuration(legitimate_users_tweets)],axis=1)

In [ ]:
data_set_polluters.insert(0,"Label",1)
data_set_legitimate.insert(0,"Label",0)

In [ ]:
data_set=pd.concat([data_set_polluters,data_set_legitimate])

Prepocessing

Replace Missiing Values (as 0)

In [ ]:
def missing(dataset):
  Replaced = dataset.replace([np.inf,-np.inf],np.nan)
  Replaced.fillna(0,inplace=True)
  return Replaced

Z-Score Standardizing

In [ ]:
def standardizing(dataset):
  df=data_set.drop(["Label"],axis=1)
  df=df.apply(zscore)
  df["Label"]=dataset["Label"]
  return df

In [ ]:
data_set=data_set.drop_duplicates()
data_set=missing(data_set)
data_set=standardizing(data_set)

In [ ]:
export_csv = data_set.to_csv (r'dataset.csv', index = None, header=True)

In [ ]:
X=data_set.drop(['Label'],axis=1)
X=X.values
Y=data_set["Label"].values 
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

Random Forest

In [ ]:
#Random Forest
clf_RF = RandomForestClassifier(n_estimators=40,oob_score=True)
clf_RF.fit(x_train, y_train)
y_pred_RF = clf_RF.predict(x_test)

SVM

In [ ]:
#SVM
clf_SVM = svm.SVC()
clf_SVM.fit(x_train, y_train)
y_pred_SVM = clf_SVM.predict(x_test)

AdaBoost

In [ ]:
#AdaBoost
clf_ada = AdaBoostClassifier(n_estimators=40, random_state=40)
clf_ada.fit(x_train, y_train)
y_pred_ada = clf_ada.predict(x_test)

Evaluation

In [ ]:
def compute_precision(detection_results, labels_list):
  precision = precision_score(detection_results, labels_list)
  print("Precision: %0.4f" % precision)

def compute_recall(detection_results, labels_list):
  recall = recall_score(detection_results, labels_list)
  print("Recall: %0.4f" % recall)

def compute_f1score(detection_results, labels_list):
  f1 = f1_score(detection_results, labels_list)
  print("F1 Score: %0.4f" % f1)

In [ ]:
compute_precision(y_test, y_pred_RF)
compute_recall(y_test, y_pred_RF)
compute_f1score(y_test, y_pred_RF)

Precision: 0.9243
Recall: 0.9252
F1 Score: 0.9247


In [ ]:
compute_precision(y_test, y_pred_SVM)
compute_recall(y_test, y_pred_SVM)
compute_f1score(y_test, y_pred_SVM)

Precision: 0.8901
Recall: 0.8574
F1 Score: 0.8734


In [ ]:
compute_precision(y_test, y_pred_ada)
compute_recall(y_test, y_pred_ada)
compute_f1score(y_test, y_pred_ada)

Precision: 0.9216
Recall: 0.9066
F1 Score: 0.9140
